![car](car.jpg)

Insurance companies invest a lot of [time and money](https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf) into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [3]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit

# Read in dataset
cars = pd.read_csv("car_insurance.csv")

import pandas as pd
from statsmodels.formula.api import logit

# Check for missing values in 'credit_score' and 'annual_mileage' and fill with the mean if necessary
if cars["credit_score"].isnull().sum() > 0:
    cars["credit_score"].fillna(cars["credit_score"].mean(), inplace=True)

if cars["annual_mileage"].isnull().sum() > 0:
    cars["annual_mileage"].fillna(cars["annual_mileage"].mean(), inplace=True)

# Empty list to store models
models = []

# Define feature columns by excluding 'id' and 'outcome'
feature_columns = cars.drop(columns=["id", "outcome"]).columns

# Fit logistic regression models for each feature and store the models
for feature in feature_columns:
    model = logit(f"outcome ~ {feature}", data=cars).fit(disp=0)  # disp=0 suppresses output
    models.append(model)

# List to store accuracies
accuracies = []

# Loop through each model and calculate its accuracy
for idx, model in enumerate(models):
    # Compute confusion matrix for each model
    confusion_matrix = model.pred_table()
    
    # Extract values from the confusion matrix
    tn, fp = confusion_matrix[0, 0], confusion_matrix[0, 1]
    fn, tp = confusion_matrix[1, 0], confusion_matrix[1, 1]
    
    # Calculate accuracy: (true positives + true negatives) / total samples
    accuracy = (tn + tp) / (tn + tp + fn + fp)
    accuracies.append(accuracy)

# Identify the best feature based on the highest accuracy
best_feature_idx = accuracies.index(max(accuracies))
best_feature = feature_columns[best_feature_idx]
best_accuracy = max(accuracies)

# Create a DataFrame for the best feature and its accuracy
best_feature_df = pd.DataFrame({
    "best_feature": [best_feature],
    "best_accuracy": [best_accuracy]
})

# Display the best feature and its accuracy
best_feature_df

,best_feature,best_accuracy
0,driving_experience,0.7771
